In [2]:
import json
import uuid
import requests

## Test embedding, chat

##### Embedding

In [26]:

url = "http://localhost:8000/embedding"
r = requests.post(url, json={"admin_password": "valory_algover"})
r.raise_for_status()
r.json()

{'ok': True, 'message': 'Embedding request received'}

In [27]:
from tenacity import (
    AsyncRetrying,
    before_sleep_log,
    retry,
    retry_if_exception_type,
    stop_after_attempt,
    wait_exponential,
)

##### chat

In [36]:
# Chatting over the embedding above. Since no `memory_id` is provided, the agent will assume this is a new conversation
id_ = uuid.uuid4().hex
payload = {
    "id": id_,
    "question": "What is daios?",
}

url = "http://localhost:8000/chat"
r = requests.post(url, json=payload)
r.raise_for_status()
r = r.json()
mem_id = r["memory_id"]


In [37]:
get_url = f"http://localhost:8000/get/{id_}"

r = requests.get(get_url)
r.raise_for_status()
r = r.json(); r

[{'id': '10467ca444a24624ba32ae9fe156d81b',
  'context_id': 'b19b850793774c8785c5ee916946b778',
  'memory_id': '5d89536018fc4968b9b61c0cb6cd56ed',
  'question': 'What is daios?',
  'modified_question': None,
  'response': 'Based on the context provided, DAIOS refers to the Decentralized AI Operating System. It is an extension of the Open AEA (Agent Environment Architecture) framework developed by Valory. The DAIOS framework aims to build a decentralized operating system for artificial intelligence, integrating blockchain, messaging, and other technologies.',
  'chat_history': [{'role': 'user', 'content': 'What is daios?'},
   {'role': 'assistant',
    'content': 'Based on the context provided, DAIOS refers to the Decentralized AI Operating System. It is an extension of the Open AEA (Agent Environment Architecture) framework developed by Valory. The DAIOS framework aims to build a decentralized operating system for artificial intelligence, integrating blockchain, messaging, and other te

In [38]:
# since a `memory_id` is provided, the agent will use the memory to generate a follow-up response
id_ = uuid.uuid4().hex
payload = {
    "id": id_,
    "context_id": emb_id_,
    "memory_id": mem_id,
    "question": "does it provide a platform?",
}

url = "http://localhost:8000/chat"
r = requests.post(url, json=payload)
r.raise_for_status()
r.json()

{'id': '877130f8a6da42ec99b6d19093b39c9c',
 'memory_id': '5d89536018fc4968b9b61c0cb6cd56ed'}

In [39]:
get_url = f"http://localhost:8000/get/{id_}"

r = requests.get(get_url)
r = r.json()
r

[{'id': '877130f8a6da42ec99b6d19093b39c9c',
  'context_id': 'b19b850793774c8785c5ee916946b778',
  'memory_id': '5d89536018fc4968b9b61c0cb6cd56ed',
  'question': 'does it provide a platform?',
  'modified_question': 'Does the DAIOS framework provide a platform?',
  'response': 'Yes, the DAIOS framework aims to provide a platform for building a Decentralized AI Operating System.',
  'chat_history': [{'role': 'user', 'content': 'What is daios?'},
   {'role': 'assistant',
    'content': 'Based on the context provided, DAIOS refers to the Decentralized AI Operating System. It is an extension of the Open AEA (Agent Environment Architecture) framework developed by Valory. The DAIOS framework aims to build a decentralized operating system for artificial intelligence, integrating blockchain, messaging, and other technologies.'},
   {'role': 'user', 'content': 'Does the DAIOS framework provide a platform?'},
   {'role': 'assistant',
    'content': 'Yes, the DAIOS framework aims to provide a plat

##### Bad requests are handled within the http_server

In [40]:
# non-existent memory
id_ = uuid.uuid4().hex
payload = {
    "id": id_,
    "context_id": "bf68b34c19964a73ba03c9779ef70fa9",
    "memory_id": "87b9426ab1f84ad689e0eb29e350db83",
    "question": "What is daios?",
}

url = "http://localhost:8000/chat"
r = requests.post(url, json=payload)
r.raise_for_status()
print(r.status_code)


HTTPError: 400 Client Error: context_id bf68b34c19964a73ba03c9779ef70fa9 does not exist for url: http://localhost:8000/chat

In [13]:
# non-existent context
id_ = uuid.uuid4().hex
payload = {
    "id": id_,
    "context_id": "bf68b34c19964a73ba03c9779ef70fa9",
    "question": "What is daios?",
}

url = "http://localhost:8000/chat"
r = requests.post(url, json=payload)
r.raise_for_status()
print(r.status_code)


HTTPError: 400 Client Error: context_id bf68b34c19964a73ba03c9779ef70fa9 does not exist for url: http://localhost:8000/chat